# [모듈 1.0] 컨피그 파일 생성

이 노트북은 아래와 같은 설정 파일을 생성 합니다. 이후 노트북에서 설정 파일을 로딩해서 사용 합니다.
- Code Pipeline Series (CodeCommit, CodeBuild, CodePipeline)
- SageMaker Pipeline 



# 1. 환경 확인

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
import sagemaker
import boto3
sys.path.append('./src')



### 입력 파일을 위한 변수는 기존에 처리가 되어야 합니다.

# 2.역할 정의
## 2.1. 아래 Code Build 역할 ARN 을 넣어 주세요. 아래를 지우고 넣습니다.¶
- build_service_role: 은 아래와 미리 생성이 됨
    -  "arn:aws:iam::XXXXXX:role/codebuild"

In [ ]:
# code_build_service_arn = "arn:aws:iam::XXXXXXXX:role/codebuild"
code_build_service_arn = "arn:aws:iam::057716757052:role/codebuild-gsmoon"

## 2.2. Code Pipeline Role 의 ARN 을 넣어 주세요. 아래를 지우고 넣습니다.
##### pipeline_role_name = "arn:aws:iam::XXXXXX:role/code-pipeline"


In [ ]:
# code_pipeline_role_arn = "arn:aws:iam::XXXXXXXX:role/code-pipeline"
code_pipeline_role_arn = "arn:aws:iam::057716757052:role/code-pipeline-gsmoon"


# 3. Code Pipeline 시리즈 파라미터 세팅

## 3.1. 변수 설정 및 dict 로 저장

In [ ]:
# 기본 정보
region = sagemaker.Session().boto_region_name
account_id = boto3.client('sts').get_caller_identity().get('Account')
project_prefix = "CodePipeline-Serving-NCF"
bucket = sagemaker.Session().default_bucket()

# Codecommit Repo Name
serving_code_repo_name = "ncf-serving-repo"

# Code Build Project 이름
code_build_project_name = "ncf-serving-sm-pipeline"

# code pipeline project 변수
code_pipeline_name = "ncf-serving-code-pipeline"
branch_name = 'master'
model_package_group_name = "NCF-Model-CodePipeline"
endpoint_name = "ncf-codepipeline-endpoint"

# 최종 json 파일 경로
code_pipeline_serving_config_json_path = f'src/code_pipeline_serving_config.json'


In [ ]:
code_pipeline_dict = {
    "code_pipeline_role_arn" : code_pipeline_role_arn,
    "code_build_service_arn" : code_build_service_arn,    
    "project_prefix" : project_prefix,
    "region" : region,        
    "account_id" : account_id,
    "serving_code_repo_name": serving_code_repo_name,    
    "code_build_project_name" : code_build_project_name,
    "bucket" : bucket,  
    "code_pipeline_name" : code_pipeline_name,  
    "model_package_group_name" :  model_package_group_name,        
    "endpoint_name" :  endpoint_name,    
    "branch_name" :  branch_name
}

## 3.2. 설정 딕션너리 Json 파일로 저장 후 테스트를 위해 로딩

In [ ]:
from common_utils import save_json, load_json

save_json(code_pipeline_serving_config_json_path, code_pipeline_dict)
code_pipeline_train_dict = load_json(code_pipeline_serving_config_json_path)


In [ ]:
import json
print (json.dumps(code_pipeline_train_dict, indent=2))

# 4. SageMaker Pipeline 파라미터 세팅

## 4.1. 변수 설정 및 dict 로 저장

In [ ]:

endpoint_instance_type = 'ml.g4dn.xlarge'
sm_pipeline_name = "ncf-serving-code-pipeline-sm-pipeline"
bucket = sagemaker.Session().default_bucket()
ModelApprovalStatus="Approved"
sm_pipeline_serving_config_json_path = f'src/sm_pipeline_serving_config.json'


In [ ]:
sm_train_pipeline_dict = {
    "sm_pipeline_name" : sm_pipeline_name,
    "endpoint_instance_type" : endpoint_instance_type,    
    "ModelApprovalStatus" :  ModelApprovalStatus,    
    "bucket" : bucket,            
}

## 4.2. 설정 딕션너리 Json 파일로 저장 후 테스트를 위해 로딩

In [ ]:
from common_utils import save_json, load_json

save_json(sm_pipeline_serving_config_json_path, sm_train_pipeline_dict)
sm_pipeline_train_dict = load_json(sm_pipeline_serving_config_json_path)
print (json.dumps(sm_pipeline_train_dict, indent=2))

# 5. Config 파일 복사

## 5.1. code_pipeline_train_config_json 파일

In [ ]:
source_path = code_pipeline_serving_config_json_path
target_path = 'codecommit/pipelines/ncf/src/'

! cp {source_path} {target_path}

## 5.2. sm_pipeline_train_config_json 파일

In [ ]:
source_path = sm_pipeline_serving_config_json_path
target_path = 'codecommit/pipelines/ncf/src/'

! cp {source_path} {target_path}

# 6. 변수 저장

In [ ]:
%store code_pipeline_serving_config_json_path
%store sm_pipeline_serving_config_json_path